<a href="https://colab.research.google.com/github/openingsound/Hands-On_Machine_Learning/blob/master/Batch_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch Normalization

## Stochastic Gradient Descent

* 문제점
  * layer를 여러층으로 쌓음으로 서 발생함
  * 하이퍼 파라미터들의 초기값 설정을 굉장히 신중하게 해줘야 한다.
  * 이전 네트워크에서의 파라미터 변화는 다음 네트워크를 거치며 변화량이 점점 증폭된다. 
  

## 개념
* NN의 입력 분포 변환는 일어나지 않는것이 좋다. 
  * NN 파라미터들이 그 분포에 새로 적응해야 하기 때문!
* 학습을 원활하게 하기 위해 효율적인 입력 분포는 일정하게 유지되는 입력 분포
* 네트워크를 여러 층을 쌓은 경우에 하위 네트워크에 대한 입력 분포의 효유성에도 동일
  * 앞단의 layer의 output의 분포도 일정하게 유지하는 것이 좋다. 

* 또 다른 문제점
  * sigmoid를 사용할 경우 절대값이 일정 수준이상(6이상) 으로 큰 입력에 대해서는 grad가 0 이된다. 
    * x의 분포가 0 으로부터 멀어진다면 Saturation이 됨(Saturated Regime)

## Internal Covariate Shift
* Covariate shift
  * train data와 test data의 data distribution이 다른 현상

*  네트워크의 학습 도중에 파라미터의 변화로 인한 네트워크 Activation(출력)들의 분포 변화
* 각각의 layer들의 input distribution이 consistent하지 않은 현상을 internal convariate shift라고 정의한다. 
* 학습 효율을 높이기위해서는 이를 줄요야 한다. 
* 입력값들이 Whitening(평균이 0, Unit Variance,입력값이 서로 Decorrelated)된다면 NN이 훨씬 빠르게 수렴 할 것이다. 또한 모든 레이어들이 같은 Whitening방식을 공유한다면 훨씬 이득을 가질 수 있다. 



## Batch Normaliztoin의 목적
파라미터 업데이트 이후마다 트레이닝셋 전체의 분석이 필요하지 않을 뿐 아니라 미분도 가능해서 Backpropagation을 구하는것이 용이한 Normalization lr을 맘편히 증가 시키자!

## Navie approach: Whitening
이 논문의 목표는 internal covariate shift를 줄이는 것이다.

* input distribution을 zero mean, unit variance를 가지는 normal distribution으로 normalize 시키는 것으로 문제를 해결할 수 있고 이를 whitening이라고 한다. 
* naive한 방법의 2가지 문제점
  * inverse의 square root를 계산해야 하기 때문에 필요한 계산량이 많다.
  * mean과 variance 세팅은 어떻게 할 것인가? 전체 데이터를 기준으로 mean variance를 training마다 계산하면 계산량이 많이 필요하다.

## 미니배치의 Statistics를 이용
* 위의 문제를 해결하기 위한 2가지 접근법
  * 각 차원들이 independent하다고 가정하고 각 차우너별로 따로 estimate하고 대신 표현형을 더 풍성하게 해 줄 linear transform도 학습한다.
  * 전체의 mean과 var대신 현재 batch 에서만 구한다음 inference할때만 real mean/var을 구한다. 
* 각각의 스칼라 Feature들을 독립적으로 정규화 하는 방식으로 진행된다. 
  * 각가의 Feature들의 mean 및var를 0과 1로 정규화 
  * $\gamma,  \beta$(std, mean)을 도입하여 모델이 학습이 되어갈때 같이 학습이 이뤄 지며 본래 모델의 Representation Power(복잡한 함수를 표현할 수 있는 힘)을 유지시키는 역할을 한다. 
* 미니배치 단위에서 정규화가 수행된다. 
  *  미니배치를 Stochastic하게 샘플링하여 정규화를 수행하게 된다. 
  * 각각의 미니배치는 각각의 Activation에 대하여 Mean 및 Variance를 추정하는데 사용된다. 
  * 이러한 방식을 통해서 정규화에 사용되는 Statistics가 Backpropagation에 활용될 수 있게 된다.

### Caution_
* 각 차원들의 Activation들을 독립적이라고 가정하고 각각의 Activation들 사이의 Joint Covariance를 고려하지 않는다
  * 단 이렇게 correlation을 무시하고 학습하는 경우 각가의 관계가 중요한 경우 제대로 되지 못한 학습을 하게 될 수도 있으므로 scaling과 shifting을 포함하는 $\gamma,  \beta$를 차원마다 학습 시킨다. 
* 파라미터 $\gamma,  \beta$로 Scale 및 Shift를 수행하여 출력값을 내보내게 된다. 이 때 $\gamma,  \beta$는 Backpropagation을 통해서 학습이 된다.  



### Batch Normalization의 특징

* 트레이닝을 위한 미니배치 단위에서 수행
* 미니배치 내의 한 Example 내에서의 Activation들은 각각 독립적이라고 가정
* 독립적인 각각의 Activation들은 정규화를 위하여 미니배치 내의 Example들의 Statistics를 이용

## 레이어 트레이닝
*  Activation의 Mean과 Variance를 미니배치 내에서 추정을 하여 Activation들을 각각 정규화를 시킨다.

## 레이어의 인퍼런스
*  트레이닝에서는 Activation의 정규화 과정에서 미니배치와의 Dependency를 고려하지만 인퍼런스에서도 이렇게 된다면 미니배치의 세팅에 따라서 결과가 달라지게 된다. 
   * 고정된 mean과 var필요
   * Sample mean과 Sample var를 이용하여 moving Avr  